## 영화리뷰 분석
---
- IMDB 데이터 활용

In [1]:
from tensorflow.keras.datasets.imdb import load_data
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
import numpy as np

## [1] 데이터 준비

In [2]:
# 단어사전(voca)에서 빈도가 높은 500개까지만 사용하도록 설정
(X_train, y_train), (X_test, y_test) = load_data(num_words=500)

In [3]:
print(f'imDB 정보 \nX_train : {X_train.shape}  y_train : {y_train.shape}')
print(f'X_test  : {X_test.shape}  y_test  : {y_test.shape}')

imDB 정보 
X_train : (25000,)  y_train : (25000,)
X_test  : (25000,)  y_test  : (25000,)


In [4]:
for idx in range(3):
    print(f'[{idx} - {y_train[idx]} : {len(X_train[idx])}]\n{X_train[idx]}')
    print('-----------------------------------------')

# 이미 다 처리된 데이터
# -> 길이 맞춰 자르고, 채워놓는 작업만 하면 되는 상태
# 1번쨰 리뷰는 218개의 토큰으로 이뤄짐

[0 - 1 : 218]
[1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 2, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
-----------------------------------------
[1 - 0 : 189]
[1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 134, 26, 4, 2, 8, 118, 2, 14, 394, 20

## [2] 학습용 데이터 변환
---
- keras에서 이미 대부분 전처리가 끝난 데이터
- 길이 조정  ==>  패딩(Padding)

In [5]:
y_test

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [6]:
# [2-1] 데이터별 길이 결정


In [7]:
# 각각의 길이를 알아야 패딩할 때, 최대 길이를 정할 수 있음
# -> 그래야 숫자의 분포를 보고 어디를 자르는게 유리할지 확인
X_train

array([list([1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 2, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 134, 26, 4, 2, 8, 118, 2, 14, 394, 20, 13, 119, 2, 189, 102, 5, 207, 110, 2, 21, 

In [8]:
X_train

array([list([1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 2, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 134, 26, 4, 2, 8, 118, 2, 14, 394, 20, 13, 119, 2, 189, 102, 5, 207, 110, 2, 21, 

In [9]:
tokenizer = Tokenizer()

In [10]:
last_value = len(tokenizer.word_index) + 1 # 단어 집합의 크기보다 1 큰 숫자를 사용
print(last_value)

1


### [2-2] 데이터별 길이 변경

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 가장 긴 것에 길이를 맞춰주기 위해, '0'으로 채워진 것을 볼 수 있음
padded_Xtr = pad_sequences(X_train, padding='post')
# padded_ytr = pad_sequences(y_train)
padded_Xte = pad_sequences(X_test) #, padding='post')
# padded_yte = pad_sequences(y_test)

print(type(padded_Xtr), padded_Xtr, sep='\n')
print(type(padded_Xte), padded_Xte, sep='\n')

<class 'numpy.ndarray'>
[[  1  14  22 ...   0   0   0]
 [  1 194   2 ...   0   0   0]
 [  1  14  47 ...   0   0   0]
 ...
 [  1  11   6 ...   0   0   0]
 [  1   2   2 ...   0   0   0]
 [  1  17   6 ...   0   0   0]]
<class 'numpy.ndarray'>
[[  0   0   0 ...  14   6   2]
 [  0   0   0 ... 125   4   2]
 [  0   0   0 ...   9  57   2]
 ...
 [  0   0   0 ...  21   2   2]
 [  0   0   0 ...   2   7 470]
 [  0   0   0 ...  34   2   2]]


In [12]:
# categoriclal
from tensorflow.keras.utils import to_categorical

train_seq = pad_sequences(X_train, maxlen=150)
# train_seq.shape, train_seq[2], train_seq[2].shape
print(train_seq.shape, train_seq, sep='\n')

(25000, 150)
[[ 12  16  43 ...  19 178  32]
 [  4 249 126 ...  16 145  95]
 [  0   0   0 ...   7 129 113]
 ...
 [140   8   2 ...   4   2   2]
 [  1   2   2 ...  12   9  23]
 [194 337   7 ... 204 131   9]]


In [13]:
# 모델 만들기

In [14]:
# train, test 같이 해줄 것
train_oh = to_categorical(train_seq)
train_oh = to_categorical(test_seq)
train_oh.shape, train_oh[0]

NameError: name 'test_seq' is not defined

### [2-3] 훈련/테스트 데이터 준비

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train 과 val만 
X_train, X_val, y_train, y_val = train_test_split(train_oh, y_train,
                                                 test_size=0.2, random_state=42)

# ----
print(f'X_train : {X_train.shape}  y_train : {y_train.shape}')
print(f'X_test  : {X_test.shape}  y_test  : {y_test.shape}')

X_train, test_seq, X_val
y_train, y_test, y_val

### [3] 모델
---
- 목표 : 영화 리뷰 데이터 학습 후, 사용자가 입력하는 리뷰가 긍정/부정 분류
- 학습모델 : RNN 순환신경망
- 입력 : 500
- 출력 : 1 => 이진 분류

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

### [3-1] 모델 구성
---
- 입력층 : 노드 n개, 타임스탬프수, 피처수   => SimpleRNN
- 출력층 : 노드 1개, sigmoid              => dense  
  

* 토큰 번호 80000만개 중, 1-500개만 쓰겠다는 것 (빈도 수로 끊어) - VOCA크기
* 150개라는 것은 문장 하나를 나타내기 위해 쓰이는 토큰 수
* 타임스탬프 : 하나의 문장을 구성하는 토큰 수

In [ ]:
model = Sequential()
# input_shape=(타임스템프크기, )           
# 한 문장을 구성하는 100개의 토큰, # 단어사전 안에 있는 것 자른 것, 샘플의 갯수에는 영향xz
model.add(SimpleRNN(units=9, input_shape=(100, 500), return_sequences=True)) # activation='tanh'
model.add(SimpleRNN(5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam', metrics=['acc'])

### [3-2] 모델 생성
---
- 손실함수, 최적화방법, 평가항목 => complile()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer = 'adam'
              metrics = 'accuracy')

### [4] 모델 학습
---
- train 데이터와 타겟으로 학습 진행
- epoch마다 검증 진행 => validation_data, validation_data
- 학습 중 이벤트 처리 => callbacks=[]
- 학습 횟수 조절 => epochs
- 배치사이즈 조절 => batch_size (기:32) 

In [ ]:
model.fit(X_train, y_train)
model.fit(train_x, train_y, epochs=100, batch_size=16)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)

In [ ]:
# 학습 되는 과정 시각화
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['loss'], label='loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.show()

### [5] 평가

## Embedding RNN Model
---
- One-Hot-Encoding 입력 데이터의 많은 소모에 대한 문제 해결
- 토큰을 밀집형태의 벡터로 변경 ==> Word Embedding
- Embedding Layer 적용
    * 입력 차원 - 토큰의 크기. 즉, 단어 사전의 크기
    * 출력 차원 - 줄여서 생성하는 벡터 크기
    * 입력층일 경우 - input_dim = 타임스탬프. 즉, 하나의 문장을 구성하는 토큰 수   

In [ ]:
from tensorflow.keras.layers import Embedding

In [ ]:
model2 = Sequential()

# One-Hot-Encoding의 거대한 데이터 =>밀집형태 벡터로 변환
model2.add(Embedding(500, 20, input_length=100))

# 500개 짜리를 20개로 줄임 => 가중치(w)가 20개 생기는 것
# 각각의 히든 스테이트의 가중치(w)가 10개 생기는 것
model2.add(SimpleRNN(10))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

In [ ]:
plot_model(model2, show_shapes=True)